In [1]:
import numpy as np
from sklearn.decomposition import PCA
import scipy.io as sio
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn import preprocessing
import os
import random
from random import shuffle
# from skimage.transform import rotate
import scipy.ndimage
from spectral import *

In [ ]:
path = 'predata'
# Global Variables
# The number of principal components to be retained in the PCA algorithm, 
# the number of retained features  n
numComponents = 30
# Patches windows size
windowSize = 5
# The proportion of Test sets
testRatio = 0.50

In [2]:
#  load the Indian pines dataset which is the .mat format
def loadIndianPinesData():
    data_path = 'E:\Eric_HSI\hyperspectral_datasets'
    data = sio.loadmat(os.path.join(data_path, 
                      'Indian_pines_corrected.mat'))['data']
    labels = sio.loadmat(os.path.join(data_path, 
                        'Indian_pines_gt.mat'))['groundT']
    
    return data, labels

In [3]:
# data, labels = loadIndianPinesData()
# data.shape, labels.shape  # ((145, 145, 200), (145, 145))

In [4]:
#  apply PCA preprocessing for data sets
def applyPCA(X, numComponents=75):
    newX = np.reshape(X, (-1, X.shape[2]))
    print(newX.shape)
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
    return newX, pca

In [5]:
# newX, pca = applyPCA(data, numComponents=30)

In [6]:
# newX.shape   # (145, 145, 30)

In [7]:
#  pad zeros to dataset, 此行代码的作用是在图像四周加了两行的0，原因是因为下面create patches的过程中，会调用一个像素四周的两行，24个像素值，在图像四周补零，会更加充分的利用仅有的像素信息，增加像素正确分类的几率。margin = 2 = (windowSize-1) /2
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2 * margin, 
                     X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + 
         y_offset, :] = X
    return newX

In [8]:
#  create Patches for dataset
def createPatches(X, y, windowSize=5, removeZeroLabels = True):
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)  # （149,149,30）
    # split patches, 创建一个想要的形状大小的矩阵即（像素总数，patches大小，patches大小，channels），（像素总数）
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    
    # 从整幅图像中，将每一个像素结合边上的5*5的像素依次分别取出来
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]   
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

In [9]:
# newX, labels = createPatches(newX, labels, windowSize=5, removeZeroLabels = True)

In [10]:
# newX.shape, labels.shape   # ((10249, 5, 5, 30), (10249,))

- sk-learn中提StratifiedShuffleSplit()提供分层抽样功能，确保每个标签对应的样本的比例
- 参数说明
    - n_splits：是将训练数据分成train/test对的组数，可根据需要进行设置，默认为10
    - test_size和train_size： 是用来设置train/test对中train和test所占的比例。例如：
        1. 提供10个数据num进行训练和测试集划分
        2. 设置train_size=0.8 test_size=0.2
        3. train_num=numtrain_size=8 test_num=numtest_size=2
     - random_state：随机数种子，和random中的seed种子一样，保证每次抽样到的数据一样，便于调试

In [11]:
# split data to Train and Test Set，将数据集划分成15组train and test，并且保证每个test中都有相同的类别
# StratifiedShuffleSplit：在划分训练集和测试集的过程中，分成多组后，保证每个划分的数据集的标签比例相同
def splitTrainTestSet(X, y, classnum=15, testRatio=0.50):
#     X_train, X_test, y_train, y_test = train_test_split(X, y, 
#                                test_size=testRatio, random_state=345, stratify=y)
    ss=StratifiedShuffleSplit(n_splits=classnum, test_size=testRatio, 
                              train_size=1-testRatio, random_state=0)   # 分成15组
    
    for train_index, test_index in ss.split(X, y):
        print("TRAIN:", len(train_index), "TEST:", len(test_index))
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
    return X_train, X_test, y_train, y_test

In [12]:
# X_train, X_test, y_train, y_test = splitTrainTestSet(newX, labels, classnum=15, testRatio=0.50)

In [13]:
# X_train.shape, X_test.shape, y_train.shape, y_test.shape  # ((5124, 5, 5, 30), (5125, 5, 5, 30), (5124,), (5125,))

## 过采样和欠采样都来源于信号处理
- 在信号处理中
1. 采样频率高于信号最高频率的两倍，这种采样被称为过采样。
2. 采样频率低于信号最高频率的两倍，这种采样被称为欠采样。
- 在图像分类中

- 在分类问题中，有存在正反例数目差异较大的情况，这种情况叫做类别不平衡。
- 针对这种问题，解决方式主要有3种：假设正例数量大，反例数目极小。
    1. 减少正例的数量，使得数据平衡，再进一步分类，这种情况属于“欠采样”；
    2. 增加反例的数目平衡数据，再分类，这种称为“过采样”；
    3. 阈值移动方法，借用一个公式加入决策过程，提高调解不平衡性

In [14]:
# over sample
# np.unique(y, return_counts=True)：返回去重后的数组y和y中每一个元素出现的次数
def oversampleWeakClasses(X, y):
    uniqueLabels, labelCounts = np.unique(y, return_counts=True)  # 得到不重复的标签，标签总数ndarray
    print("uniqueLabels {} \nlabelCounts {}".format(uniqueLabels, labelCounts))
    maxCount = np.max(labelCounts)
    labelInverseRatios = maxCount / labelCounts    # 应该是不平衡性的度量指标
    print("labelInverseRatios", labelInverseRatios)
    
    # repeat for every label and concat
    # X(5124, 5, 5, 30) y(5124)
    # 这两行貌似没有什么卵用，是用来验证这种方法可行的，将标签为0的重复了round(labelInverseRatios[0]次
    newX = X[y == uniqueLabels[0], :, :, :].repeat(round(labelInverseRatios[0]), axis=0)
    newY = y[y == uniqueLabels[0]].repeat(round(labelInverseRatios[0]), axis=0)
    print(newX.shape, newY.shape)

    # 下面是对上面验证好的代码的使用，使用循环将每个数目较少标签都进行过采样
    for label, labelInverseRatio in zip(uniqueLabels[1:], labelInverseRatios[1:]):
        cX = X[y == label,:,:,:].repeat(round(labelInverseRatio), axis=0)
        cY = y[y == label].repeat(round(labelInverseRatio), axis=0)
        # 每重复一次，将他们连接在原始中
        newX = np.concatenate((newX, cX))
        newY = np.concatenate((newY, cY))
    
    np.random.seed(seed=42)
    # 随机排列序列，因为每次加上的都是相通的便签，噪声相通便签的大量重复出现，这里随机打乱顺序
    rand_perm = np.random.permutation(newY.shape[0])
    print(rand_perm.shape)
    # 保证标签和数据匹配
    newX = newX[rand_perm, :, :, :]
    newY = newY[rand_perm]
    return newX, newY

In [15]:
# newX, newY = oversampleWeakClasses(X_train, y_train)  
# newX.shape, newY.shape  # ((19788, 5, 5, 30), (19788,))

In [16]:
# 这里验证用True 和 False 也可以作为索引
# X[y == uniqueLabels[0], :, :, :].repeat((labelInverseRatios[0]), axis=0)
XX = np.arange(10)
zz = np.arange(15)
yy = np.arange(10)
print(type(XX))
print(yy == zz[0])
aa = [True, False, False, False, False, False, False, False, False, False]
print(XX[aa])
print(XX[yy == zz[0]])
XX = XX[yy == zz[0]].repeat(20)
XX

<class 'numpy.ndarray'>
[ True False False False False False False False False False]
[0]
[0]


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [17]:
#  Augment Data
def AugmentData(X_train):
    for i in range(int(X_train.shape[0]/2)):
        patch = X_train[i,:,:,:]
        num = random.randint(0,2)
        if (num == 0):
            flipped_patch = np.flipud(patch)
        if (num == 1):
            flipped_patch = np.fliplr(patch)
        if (num == 2):
            no = random.randrange(-180,180,30)
            flipped_patch = scipy.ndimage.interpolation.rotate(patch, 
                            no,axes=(1, 0), reshape=False, output=None, 
                            order=3, mode='constant', cval=0.0, prefilter=False)
        
    patch2 = flipped_patch
    X_train[i,:,:,:] = patch2
    
    return X_train

In [18]:
# newX = AugmentData(newX)  
# newX.shape    # (19788, 5, 5, 30)

In [19]:
#  standartize
def standartizeData(X):
    newX = np.reshape(X, (-1, X.shape[0]))
    scaler = preprocessing.StandardScaler().fit(newX)  
    newX = scaler.transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1],X.shape[2],X.shape[3]))
    return newX, scaler

In [20]:
# newX = standartizeData(newX)
# X_test = standartizeData(X_test)

In [21]:
# save Preprocessed Data to file
def savePreprocessedData(path, X_trainPatches, X_testPatches, y_trainPatches, 
                         y_testPatches, windowSize, wasPCAapplied = False, 
                         numPCAComponents = 0, testRatio = 0.25):
    
    data_path = os.path.join(os.getcwd(), path)
    if wasPCAapplied:
        with open(os.path.join(data_path, "X_train_WS_") + str(windowSize) + 
                  "_PCA_" + str(numPCAComponents) + "_testRatio_" + str(testRatio) + 
                  ".npy", 'bw') as outfile:
            np.save(outfile, X_trainPatches)
        with open(os.path.join(data_path, "X_test_WS_") + str(windowSize) + 
                  "_PCA_" + str(numPCAComponents) + "_testRatio_" + str(testRatio) + 
                  ".npy", 'bw') as outfile:
            np.save(outfile, X_testPatches)
        with open(os.path.join(data_path, "y_train_WS_") + str(windowSize) + 
                  "_PCA_" + str(numPCAComponents) + "_testRatio_" + str(testRatio) + 
                  ".npy", 'bw') as outfile:
            np.save(outfile, y_trainPatches)
        with open(os.path.join(data_path, "y_test_WS_") + str(windowSize) + 
                  "_PCA_" + str(numPCAComponents) + "_testRatio_" + str(testRatio) + 
                  ".npy", 'bw') as outfile:
            np.save(outfile, y_testPatches)
    else:
        with open(os.path.join(data_path, "preXtrainWindowSize") + str(windowSize) + 
                  ".npy", 'bw') as outfile:
            np.save(outfile, X_trainPatches)
        with open(os.path.join(data_path, "preXtestWindowSize") + str(windowSize) + 
                  ".npy", 'bw') as outfile:
            np.save(outfile, X_testPatches)
        with open(os.path.join(data_path, "preytrainWindowSize") + str(windowSize) + 
                  ".npy", 'bw') as outfile:
            np.save(outfile, y_trainPatches)
        with open(os.path.join(data_path, "preytestWindowSize") + str(windowSize) + 
                  ".npy", 'bw') as outfile:
            np.save(outfile, y_testPatches)

In [22]:
path = 'predata'
# Global Variables
# The number of principal components to be retained in the PCA algorithm, 
# the number of retained features  n
numComponents = 30
# Patches windows size
windowSize = 5
# The proportion of Test sets
testRatio = 0.50

In [23]:
# savePreprocessedData(path, newX, newY, X_test, y_test, windowSize = windowSize,
#                      wasPCAapplied=True, numPCAComponents = numComponents, testRatio = testRatio)

# 调用上面的函数！

In [24]:
# Load dataset from file and apply PCA
# X, y = loadHSIData()
X, y = loadIndianPinesData()
print('X.shape',X.shape)
print('y.shape',y.shape)
X, pca = applyPCA(X, numComponents=numComponents)
print('X.shape',X.shape)

X.shape (145, 145, 200)
y.shape (145, 145)
(21025, 200)
X.shape (145, 145, 30)


In [25]:
# Preprocess Data
XPatches, yPatches = createPatches(X, y, windowSize=windowSize)
print('XPatches',XPatches.shape)

XPatches (10249, 5, 5, 30)


In [26]:
X_train, X_test, y_train, y_test = splitTrainTestSet(XPatches, yPatches, y.max()-y.min(), testRatio)
print('X_train.shape,y_train.shape',(X_train.shape,y_train.shape))
X_train, y_train = oversampleWeakClasses(X_train, y_train)
print('X_train.shape,y_train.shape',(X_train.shape,y_train.shape))

TRAIN: 5124 TEST: 5125
TRAIN: 5124 TEST: 5125
TRAIN: 5124 TEST: 5125
TRAIN: 5124 TEST: 5125
TRAIN: 5124 TEST: 5125
TRAIN: 5124 TEST: 5125
TRAIN: 5124 TEST: 5125
TRAIN: 5124 TEST: 5125
TRAIN: 5124 TEST: 5125
TRAIN: 5124 TEST: 5125
TRAIN: 5124 TEST: 5125
TRAIN: 5124 TEST: 5125
TRAIN: 5124 TEST: 5125
TRAIN: 5124 TEST: 5125
TRAIN: 5124 TEST: 5125
TRAIN: 5124 TEST: 5125
X_train.shape,y_train.shape ((5124, 5, 5, 30), (5124,))
uniqueLabels [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.] 
labelCounts [  23  714  415  119  241  365   14  239   10  486 1227  296  103  632
  193   47]
labelInverseRatios [ 53.34782609   1.71848739   2.95662651  10.31092437   5.09128631
   3.36164384  87.64285714   5.13389121 122.7          2.52469136
   1.           4.14527027  11.91262136   1.9414557    6.35751295
  26.10638298]
(1219, 5, 5, 30) (1219,)
(19788,)
X_train.shape,y_train.shape ((19788, 5, 5, 30), (19788,))


In [27]:
X_train = AugmentData(X_train)
print('X_train',X_train.shape)

X_train (19788, 5, 5, 30)


In [29]:
# save Preprocessed Data to file
savePreprocessedData('predata', X_train, X_test, y_train, y_test, 
                     windowSize = windowSize, wasPCAapplied=True,
                     numPCAComponents = numComponents, testRatio = testRatio)